In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import argparse
import random

In [2]:
from Discriminator import Discriminator
from Generator import Generator
import torchvision.datasets
import torch.utils.data

In [3]:
ngpu = torch.cuda.device_count()
batch_size = 128
nc = 3
ngf = 64
ndf = 64
lr = 0.0002
beta1 = 0.5
image_size = 64
nz = 100
num_epochs = 5
num_workers = 2
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True)

In [5]:
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils

transforms = transforms.Compose([
    transforms.Resize(image_size),
    transforms.CenterCrop(image_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = dset.ImageFolder(root='./opt/ml/input/data/',
                           transform=transforms)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)


In [6]:
def init_weights(m):
    if isinstance(m, (nn.ConvTranspose2d, nn.Conv2d)):
        nn.init.normal_(m.weight.data, 0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [7]:
generator = Generator(nz, ngf, nc, ngpu)

generator.apply(init_weights)

discriminator = Discriminator(ngpu, nc, ndf)

discriminator.apply(init_weights)


Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2, inplace=True)
    (3): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.2, inplace=True)
    (9): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.2, inplace=True)
    (12): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1))
    (13): Sigmoid()
  )
)

In [8]:
criterion = nn.BCELoss()
G_opt = optim.Adam(params=generator.parameters(), lr=lr, betas=(0.5, 0.999))
D_opt = optim.Adam(params=discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
real_label = 1
fake_label = 0
device="cpu"

num_epochs = 1

discriminator_losses = []
generator_losses = []
iters = 0
grids = []

fake_noise = torch.randn(64, 100, 1, 1)

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader):
        #TRAIN DISCRIMINATOR
        # run real images through discriminator
        images, _ = data
        images = images.to(device)
        batch_size = images.size()[0]
        labels_real = torch.full((batch_size,), real_label, dtype=torch.float, device=device)


        D_opt.zero_grad()
        output = discriminator(images).view(-1)
        dreal_loss = criterion(output, labels_real)
        dreal_loss.backward()

        # run fake images through discriminator
        start_vectors = torch.randn(batch_size, 100, 1, 1).to(device)
        g_output = generator(start_vectors)
        d_output = discriminator(g_output.detach()).view(-1)
        labels_fake = labels_real.fill_(fake_label)
        dfake_loss = criterion(d_output, labels_fake)
        dfake_loss.backward()

        #optimize discriminator
        dloss = (dfake_loss + dreal_loss)/2
        discriminator_losses.append(dloss.item())
        D_opt.step()

        #TRAIN GENERATOR
        G_opt.zero_grad()
        d_generated_images = discriminator(g_output).view(-1)
        labels_real = labels_real.fill_(real_label)
        dgen_loss = criterion(d_generated_images, labels_real)
        generator_losses.append(dgen_loss.item())
        dgen_loss.backward()
        G_opt.step()
          
        if (i % 50 == 0):
            print(f'{epoch}, {i}, {len(dataloader)}, {dfake_loss.item()}, {dreal_loss.item()}, {dgen_loss.item()}')
  
        # if (iters % 500 == 0):
        #     with torch.no_grad():
        #         output = generator(fake_noise).detach()
        #         grids.append(output)

        iters += 1





TypeError: full() received an invalid combination of arguments - got (tuple), but expected one of:
 * (tuple of ints size, Number fill_value, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, Number fill_value, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [ ]:
# torch.save(discriminator.state_dict(), './Models/discriminator')
# torch.save(generator.state_dict(), './Models/generator')

In [ ]:
def train(args, disc, gen, dataloader, G_opt, D_opt, device, epoch):
    
    criterion = nn.BCELoss()

    # discriminator_losses = []
    # generator_losses = []
    iters = 0
    # grids = []

    # fake_noise = torch.randn(64, 100, 1, 1)

    for i, data in enumerate(dataloader):
        #TRAIN DISCRIMINATOR
        # run real images through discriminator
        images, _ = data
        images = images.to(device)
        b_size = images.size[0]
        D_opt.zero_grad()
        output = disc(images).view(-1,)
        labels_real = torch.full((b_size, ), real_label, dtype=torch.float, device=device)
        dreal_loss = criterion(output, labels_real)
        dreal_loss.backward()

        # run fake images through discriminator
        start_vectors = torch.randn(batch_size, 100, 1, 1)
        g_output = gen(start_vectors)
        d_output = discriminator(g_output.detach()).view(-1,)
        labels_fake = labels_real.fill_(fake_label)
        dfake_loss = criterion(d_output, labels_fake)
        dfake_loss.backward()

        # optimize discriminator
        # dloss = dfake_loss.item() + dreal_loss.item()
        # discriminator_losses.append(dloss)
        D_opt.step()

        #TRAIN GENERATOR
        G_opt.zero_grad()
        generated_images = generator(start_vectors)
        d_generated_images = discriminator(generated_images).view(-1, 1)
        dgen_loss = criterion(d_generated_images, labels_real)
        # generator_losses.append(dgen_loss.item())
        dgen_loss.backward()
        G_opt.step()
          
        if (i % 50 == 0):
            print(f'EPOCH: {epoch}, i: {i}, Discriminator fake loss: {dfake_loss.item()}, Discriminator real loss: {dreal_loss.item()}, Discriminator loss: {dloss},Generator loss: {dgen_loss.item()}')
  

        # if (iters % 500 == 0):
        #     with torch.no_grad():
        #         output = generator(fake_noise).detach()
        #         grids.append(output)
        
        iters += 1





    